# Mobil Coffee Clustering

This notebook performs clustering analysis on `mobil-coffee.csv` to identify customer segments based on their coffee shop visiting patterns, focusing on time spent in cafes and visit frequency.

## 1. Mount Google Drive

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from datetime import timedelta

print("Mounting Google Drive...")
drive.mount('/content/drive')

## 1b. Load Data and Initial Processing (from Drive)

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta

# IMPORTANT: Update this path to the correct location of your mobil_coffee.csv file on Google Drive!
file_path = '/content/drive/MyDrive/path/to/your/mobil_coffee.csv'
print(f"Attempting to load main data from: {file_path}")

# Define columns to load and their potential dtypes to save memory
columns_to_load = {
    'device_aid': 'str',
    'timestamp': 'str', # Load as string first, then parse
    'isim': 'str', # Cafe name
    'original_latitude': 'float32',
    'original_longitude': 'float32'
}

print(f"Loading {file_path}...")
try:
    # Attempt to load in chunks if the file is very large, or load directly
    # For now, direct load with specified dtypes and columns
    df = pd.read_csv(
        file_path, 
        sep=';', 
        usecols=columns_to_load.keys(), 
        dtype=columns_to_load,
        low_memory=False
    )
    print("CSV loaded successfully.")
except FileNotFoundError:
    print(f"Error: The file {file_path} was not found. Please check the path.")
    df = None
except Exception as e:
    print(f"Error loading CSV: {e}")
    df = None

if df is not None:
    print("Initial data sample:")
    print(df.head())
    print("\nData info:")
    df.info()

    # Convert timestamp to datetime objects
    print("\nConverting timestamp to datetime...")
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    
    # Drop rows where timestamp conversion failed
    df.dropna(subset=['timestamp'], inplace=True)
    
    # Sort data for visit identification
    print("Sorting data by device_aid, isim, and timestamp...")
    df.sort_values(['device_aid', 'isim', 'timestamp'], inplace=True)
    
    print("\nData after timestamp conversion and sorting:")
    print(df.head())
    df.info()

## 2. Identify Individual Visits and Calculate Durations

A visit is defined by consecutive pings from the same device at the same cafe. A new visit starts if the time gap between pings exceeds a threshold (e.g., 1 hour) or if the pings are on different days or at different cafe locations (though 'isim' might group different branches, using lat/lon could refine this if needed).

In [ ]:
if df is not None:
    # Define a threshold for separating visits (e.g., 1 hour)
    visit_separation_threshold = timedelta(hours=1)

    # Add a 'cafe_location_id' to distinguish between different branches of the same cafe chain if 'isim' is too general
    # For now, we can combine 'isim' with rounded lat/lon to create a more specific location ID
    # Rounding lat/lon to ~100m precision (0.001 degrees)
    df['cafe_location_id'] = df['isim'] + "_" + df['original_latitude'].round(3).astype(str) + "_" + df['original_longitude'].round(3).astype(str)
    
    print("Identifying visits...")
    # Calculate time difference between consecutive pings for the same device at the same cafe_location_id
    df['time_diff_to_prev'] = df.groupby(['device_aid', 'cafe_location_id'])['timestamp'].diff()

    # Identify the start of a new visit
    # A new visit starts if:
    # 1. It's the first ping for that device_aid at that cafe_location_id (time_diff_to_prev is NaT)
    # 2. The time_diff_to_prev exceeds the visit_separation_threshold
    # 3. The day changes (already implicitly handled by threshold if it's large enough, but can be explicit)
    df['new_visit_marker'] = (
        df['time_diff_to_prev'].isna() | 
        (df['time_diff_to_prev'] > visit_separation_threshold)
    ).astype(int)

    # Create a visit_id by taking the cumulative sum of new_visit_marker for each device_aid at each cafe_location_id
    df['visit_group_id'] = df.groupby(['device_aid', 'cafe_location_id'])['new_visit_marker'].cumsum()
    df['visit_id'] = df['device_aid'] + "_" + df['cafe_location_id'] + "_" + df['visit_group_id'].astype(str)

    print("Aggregating pings into visits...")
    # Aggregate pings into visits
    visit_data = df.groupby('visit_id').agg(
        device_aid=('device_aid', 'first'),
        cafe_location_id=('cafe_location_id', 'first'),
        isim=('isim', 'first'),
        original_latitude=('original_latitude', 'first'), # Could use mean if pings vary slightly
        original_longitude=('original_longitude', 'first'),
        visit_start_time=('timestamp', 'min'),
        visit_end_time=('timestamp', 'max'),
        num_pings=('timestamp', 'count')
    ).reset_index()

    # Calculate visit duration
    visit_data['visit_duration'] = visit_data['visit_end_time'] - visit_data['visit_start_time']

    # Filter out very short durations if they are likely noise (e.g., less than 1 minute and only 1 ping)
    # For now, let's keep all durations and analyze later.
    # A duration of 0 is possible if there's only one ping for that visit.

    # Extract hour and day of week from visit_start_time
    visit_data['visit_start_hour'] = visit_data['visit_start_time'].dt.hour
    visit_data['visit_start_day_of_week'] = visit_data['visit_start_time'].dt.day_name()

    print("\nSample of processed visit data:")
    print(visit_data.head().to_string())
    print("\nInfo for visit data:")
    visit_data.info()

    # Save this intermediate result
    # IMPORTANT: Update this path if you want to save to a different Drive location!
    output_visits_path = '/content/drive/MyDrive/path/to/your/coffee_visits_detailed.csv'
    print(f"\nSaving detailed visit data to {output_visits_path}...")
    try:
        visit_data.to_csv(output_visits_path, index=False, sep=';')
        print(f"Detailed visit data saved successfully to {output_visits_path}.")
    except Exception as e:
        print(f"Error saving detailed visit data CSV: {e}")
else:
    print("DataFrame 'df' not loaded. Cannot proceed with visit identification.")

## 3. Device-Level Feature Aggregation

Now, aggregate the visit data to create features for each unique device_aid.

In [ ]:
# IMPORTANT: Update this path to where 'coffee_visits_detailed.csv' was saved on Drive!
detailed_visits_path = '/content/drive/MyDrive/path/to/your/coffee_visits_detailed.csv'
print(f"Loading {detailed_visits_path} for device-level aggregation...")
try:
    df_visits = pd.read_csv(detailed_visits_path, sep=';')
    # Ensure correct data types after loading, especially for datetime/timedelta
    df_visits['visit_start_time'] = pd.to_datetime(df_visits['visit_start_time'])
    df_visits['visit_end_time'] = pd.to_datetime(df_visits['visit_end_time'])
    # Pandas read_csv doesn't automatically convert to timedelta; it might be a string.
    # Convert visit_duration from string (e.g., "0 days 01:23:45") to timedelta seconds for easier aggregation.
    df_visits['visit_duration_seconds'] = pd.to_timedelta(df_visits['visit_duration']).dt.total_seconds()
    print("Detailed visits data loaded successfully.")
except FileNotFoundError:
    print(f"Error: {detailed_visits_path} not found.")
    df_visits = None
except Exception as e:
    print(f"Error loading {detailed_visits_path}: {e}")
    df_visits = None

if df_visits is not None:
    print("Aggregating features per device_aid...")

    # Basic aggregations
    device_features = df_visits.groupby('device_aid').agg(
        total_visits=('visit_id', 'count'),
        total_pings=('num_pings', 'sum'),
        avg_visit_duration_seconds=('visit_duration_seconds', 'mean'),
        total_time_spent_seconds=('visit_duration_seconds', 'sum'),
        num_unique_cafes_visited=('cafe_location_id', 'nunique'),
        first_visit_date=('visit_start_time', 'min'),
        last_visit_date=('visit_start_time', 'max')
    ).reset_index()

    # Calculate overall observation period for frequency calculation
    device_features['observation_period_days'] = (device_features['last_visit_date'] - device_features['first_visit_date']).dt.days + 1 # Add 1 to avoid division by zero if first=last
    device_features['avg_visits_per_week'] = (device_features['total_visits'] / device_features['observation_period_days']) * 7
    
    # Time Slot Features (Morning, Afternoon, Evening, Night)
    df_visits['visit_start_hour'] = df_visits['visit_start_time'].dt.hour # Ensure this column exists if not carried from previous cell
    def get_time_slot(hour):
        if 6 <= hour <= 11: return 'Morning'    # 06:00 - 11:59
        elif 12 <= hour <= 17: return 'Afternoon' # 12:00 - 17:59
        elif 18 <= hour <= 23: return 'Evening'   # 18:00 - 23:59
        else: return 'Night'                      # 00:00 - 05:59
    df_visits['TimeSlot'] = df_visits['visit_start_hour'].apply(get_time_slot)
    time_slot_dummies = pd.get_dummies(df_visits.set_index('device_aid')['TimeSlot'], prefix='TimeSlot')
    time_slot_counts = time_slot_dummies.groupby('device_aid').sum()
    # Calculate rates
    for col in time_slot_counts.columns:
        time_slot_counts[col + '_rate'] = time_slot_counts[col] / device_features.set_index('device_aid')['total_visits']
        time_slot_counts.drop(columns=[col], inplace=True)
    device_features = device_features.merge(time_slot_counts, on='device_aid', how='left').fillna(0)

    # Day of Week Features
    df_visits['visit_start_day_of_week'] = df_visits['visit_start_time'].dt.day_name() # Ensure this column exists
    day_of_week_dummies = pd.get_dummies(df_visits.set_index('device_aid')['visit_start_day_of_week'], prefix='DayOfWeek')
    day_of_week_counts = day_of_week_dummies.groupby('device_aid').sum()
    # Calculate rates
    for col in day_of_week_counts.columns:
        day_of_week_counts[col + '_rate'] = day_of_week_counts[col] / device_features.set_index('device_aid')['total_visits']
        day_of_week_counts.drop(columns=[col], inplace=True)
    device_features = device_features.merge(day_of_week_counts, on='device_aid', how='left').fillna(0)

    # Convert avg_visit_duration to minutes for better interpretability
    device_features['avg_visit_duration_minutes'] = device_features['avg_visit_duration_seconds'] / 60
    device_features['total_time_spent_hours'] = device_features['total_time_spent_seconds'] / 3600
    # Drop intermediate columns like seconds, dates used for calculation if not needed for clustering
    device_features.drop(columns=['avg_visit_duration_seconds', 'total_time_spent_seconds', 'first_visit_date', 'last_visit_date', 'observation_period_days'], inplace=True, errors='ignore')

    print("\nSample of aggregated device features:")
    print(device_features.head().to_string())
    print("\nInfo for aggregated device features:")
    device_features.info()

    # Save aggregated features
    # IMPORTANT: Update this path if you want to save to a different Drive location!
    output_agg_features_path = '/content/drive/MyDrive/path/to/your/coffee_device_aggregated_features.csv'
    print(f"\nSaving aggregated device features to {output_agg_features_path}...")
    try:
        device_features.to_csv(output_agg_features_path, index=False, sep=';')
        print(f"Aggregated device features saved successfully to {output_agg_features_path}.")
    except Exception as e:
        print(f"Error saving aggregated device features CSV: {e}")
else:
    print("df_visits DataFrame not found. Cannot aggregate device features.")

## 4. Feature Preprocessing (Filtering and Scaling)

Load the aggregated device features, filter out less informative ones, and scale them for clustering.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
import matplotlib.pyplot as plt
import seaborn as sns

# IMPORTANT: Update this path to where 'coffee_device_aggregated_features.csv' was saved on Drive!
agg_features_path = '/content/drive/MyDrive/path/to/your/coffee_device_aggregated_features.csv'
print(f"Loading {agg_features_path} for filtering and scaling...")
try:
    df_agg_features = pd.read_csv(agg_features_path, sep=';')
    # Set device_aid as index, as it's the identifier, not a feature for clustering
    df_agg_features.set_index('device_aid', inplace=True)
    print("Aggregated device features loaded successfully.")
except FileNotFoundError:
    print(f"Error: {agg_features_path} not found.")
    df_agg_features = None
except Exception as e:
    print(f"Error loading {agg_features_path}: {e}")
    df_agg_features = None

if df_agg_features is not None:
    print("\nData types before filtering/scaling:")
    print(df_agg_features.dtypes)
    df_features_to_filter = df_agg_features.copy()

    # Ensure all columns are numeric for variance thresholding and correlation
    # This should be the case already based on previous steps, but as a safeguard:
    numeric_cols = df_features_to_filter.select_dtypes(include=np.number).columns
    df_features_to_filter = df_features_to_filter[numeric_cols]
    
    # 8.1. Düşük Varyanslı Özelliklerin Çıkarılması (Copied from restaurant_clustered.ipynb and adapted)
    print("\nOriginal number of features:", df_features_to_filter.shape[1])
    variance_threshold_value = 0.01 # Adjust as needed
    selector = VarianceThreshold(threshold=variance_threshold_value)
    try:
        # Ensure no NaN values before fitting VarianceThreshold
        df_features_to_filter_no_nan = df_features_to_filter.fillna(0) # Or use another imputation strategy
        selector.fit(df_features_to_filter_no_nan)
        retained_features_mask = selector.get_support()
        removed_low_variance_features = df_features_to_filter.columns[~retained_features_mask]
        df_processed_features = df_features_to_filter.loc[:, retained_features_mask].copy()
        if len(removed_low_variance_features) > 0:
            print(f"Removed {len(removed_low_variance_features)} low-variance features (threshold < {variance_threshold_value}):")
            for feature in removed_low_variance_features:
                print(f"- {feature} (Variance: {df_features_to_filter[feature].var():.4f})")
        else:
            print(f"No features removed by variance threshold (< {variance_threshold_value}).")
        print(f"Number of features after variance filtering: {df_processed_features.shape[1]}")
    except ValueError as e:
        print(f"Error during variance thresholding: {e}. Skipping.")
        df_processed_features = df_features_to_filter.copy()

    # 8.2. Yüksek Korelasyonlu Özelliklerin Çıkarılması (Copied and adapted)
    if df_processed_features.shape[1] > 1:
        print("\nStarting high-correlation feature removal...")
        corr_matrix = df_processed_features.corr().abs()
        upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        correlation_threshold = 0.90 # Adjust as needed
        features_to_drop_corr = set()
        removed_correlation_info = []
        for column in upper_triangle.columns:
            if column in features_to_drop_corr: continue
            highly_correlated_with_column = upper_triangle[upper_triangle[column] > correlation_threshold].index
            for feature in highly_correlated_with_column:
                if feature not in features_to_drop_corr:
                    features_to_drop_corr.add(feature)
                    removed_correlation_info.append(f"- '{feature}' (>{correlation_threshold*100:.0f}% vs '{column}': {corr_matrix.loc[column, feature]:.2f})")
        if len(features_to_drop_corr) > 0:
            df_processed_features.drop(columns=list(features_to_drop_corr), inplace=True)
            print(f"Removed {len(features_to_drop_corr)} highly-correlated features (threshold > {correlation_threshold}):")
            for info in removed_correlation_info:
                print(info)
        else:
            print(f"No features removed by correlation threshold (> {correlation_threshold}).")
        print(f"Number of features after correlation filtering: {df_processed_features.shape[1]}")
    elif df_processed_features.shape[1] <= 1:
        print("Skipping correlation filtering (1 or no features left).")

    # 8.3. Ölçeklendirme (Filtrelenmiş Özelliklerle)
    if df_processed_features.shape[1] > 0:
        print("\nScaling the remaining features...")
        feature_columns_for_scaling = df_processed_features.columns
        scaler = StandardScaler()
        # Ensure no NaN values before scaling
        df_processed_features_no_nan_scaling = df_processed_features.fillna(df_processed_features.mean()) # Impute with mean, or 0
        df_scaled_features_array = scaler.fit_transform(df_processed_features_no_nan_scaling[feature_columns_for_scaling])
        df_scaled_features = pd.DataFrame(df_scaled_features_array, columns=feature_columns_for_scaling, index=df_processed_features.index)
        print("Features scaled successfully.")
        print(df_scaled_features.head().to_string())
        print(f"Shape of scaled features: {df_scaled_features.shape}")
    else:
        print("No features remaining after filtering. Clustering cannot proceed.")
        df_scaled_features = None
else:
    print("Skipping feature filtering/scaling as df_agg_features was not loaded.")
    df_processed_features = None # To be used for K-Means input if scaling fails
    df_scaled_features = None

## 5. Apply K-Means Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

if 'df_scaled_features' in locals() and df_scaled_features is not None and not df_scaled_features.empty:
    inertia = []
    silhouette_scores = []
    k_range = range(2, 11) # Test K from 2 to 10
    print("\nCalculating inertia and silhouette scores for K range...")
    for k_val in k_range:
        kmeans = KMeans(n_clusters=k_val, random_state=42, n_init='auto')
        kmeans.fit(df_scaled_features)
        inertia.append(kmeans.inertia_)
        # Silhouette score can be computationally expensive for large datasets
        # If it's too slow, consider sampling: silhouette_score(df_scaled_features.sample(n=50000, random_state=42), kmeans.predict(df_scaled_features.sample(n=50000, random_state=42)))
        score = silhouette_score(df_scaled_features, kmeans.labels_)
        silhouette_scores.append(score)
        print(f"K={k_val}, Inertia: {kmeans.inertia_:.2f}, Silhouette Score: {score:.4f}")

    # Plotting Elbow Method and Silhouette Scores
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(k_range, inertia, marker='o')
    plt.title('Elbow Method for Optimal K')
    plt.xlabel('Number of Clusters (K)')
    plt.ylabel('Inertia')
    plt.xticks(k_range)
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(k_range, silhouette_scores, marker='o')
    plt.title('Silhouette Scores for Optimal K')
    plt.xlabel('Number of Clusters (K)')
    plt.ylabel('Silhouette Score')
    plt.xticks(k_range)
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    if silhouette_scores:
        optimal_k = k_range[np.argmax(silhouette_scores)]
        print(f"\nOptimal K based on highest Silhouette Score: {optimal_k}")
    else:
        print("Could not determine optimal K from silhouette scores. Defaulting to K=3 or checking Elbow plot manually.")
        optimal_k = 3 # Fallback, user should check plots

    # Apply K-Means with optimal K
    print(f"\nApplying K-Means with K={optimal_k}...")
    kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init='auto')
    cluster_labels = kmeans_final.fit_predict(df_scaled_features)
    
    # Add cluster labels to the (unscaled but filtered) aggregated features dataframe for easier interpretation
    # df_processed_features should be the one before scaling but after filtering
    df_clustered_devices = df_processed_features.copy() 
    df_clustered_devices['cluster'] = cluster_labels
    
    print("K-Means clustering applied. Cluster labels added.")
    print(df_clustered_devices.head().to_string())
    print("\nCluster sizes:")
    print(df_clustered_devices['cluster'].value_counts().sort_index())

    # Save clustered data
    # IMPORTANT: Update this path if you want to save to a different Drive location!
    clustered_output_path = '/content/drive/MyDrive/path/to/your/coffee_device_clusters.csv'
    print(f"\nSaving clustered device data to {clustered_output_path}...")
    try:
        df_clustered_devices.reset_index().to_csv(clustered_output_path, sep=';', index=False) # save device_aid from index
        print(f"Clustered device data saved successfully to {clustered_output_path}.")
    except Exception as e:
        print(f"Error saving clustered data CSV: {e}")
else:
    print("\nSkipping K-Means application as scaled features are not available or empty.")
    df_clustered_devices = None # Ensure it's defined for the next step

## 6. Cluster Interpretation and Visualization

In [ ]:
if 'df_clustered_devices' in locals() and df_clustered_devices is not None:
    print("\nCluster characteristics (mean values of features per cluster):")
    # Use df_processed_features (unscaled but filtered) for cluster_analysis if df_clustered_devices was based on it
    # Or, if df_clustered_devices already has the correct (unscaled) features:
    cluster_analysis = df_clustered_devices.groupby('cluster').mean()
    print(cluster_analysis.to_string())

    # Visualizations (adapted from restaurant_clustered.ipynb)
    # Ensure 'kmeans_final' and 'df_scaled_features' are available from the previous cell for radar chart
    if 'kmeans_final' in locals() and hasattr(kmeans_final, 'cluster_centers_') and \
       'df_scaled_features' in locals() and df_scaled_features is not None:
        
        scaled_cluster_centers = kmeans_final.cluster_centers_
        feature_names_for_radar = df_scaled_features.columns.tolist()
        num_clusters_viz = len(scaled_cluster_centers)

        angles = np.linspace(0, 2 * np.pi, len(feature_names_for_radar), endpoint=False).tolist()
        angles += angles[:1] # Close the plot

        fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
        for i in range(num_clusters_viz):
            values = scaled_cluster_centers[i].tolist()
            values += values[:1] # Close the plot
            ax.plot(angles, values, linewidth=2, linestyle='solid', label=f"Cluster {i}")
            ax.fill(angles, values, alpha=0.25)
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(feature_names_for_radar, size=8)
        ax.set_title("Radar Chart of Scaled Cluster Centers", size=16, y=1.1)
        ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
        # Save plot if needed: plt.savefig('radar_chart_coffee_clusters.png')
        plt.show()
    else:
        print("Skipping radar chart: prerequisites (kmeans_final, df_scaled_features) not met.")

    # Box Plots for key features
    # Select a few key features for box plots to avoid too many plots
    key_features_for_boxplot = [
        'avg_visit_duration_minutes',
        'total_visits',
        'num_unique_cafes_visited',
        'avg_visits_per_week',
        'total_time_spent_hours'
    ] 
    # Add some rate features if they exist after filtering
    for col in df_clustered_devices.columns:
        if '_rate' in col and len(key_features_for_boxplot) < 8: # Limit total boxplots
             if col in df_clustered_devices.columns: # Check if it survived filtering
                key_features_for_boxplot.append(col)
    
    # Filter to existing columns only
    key_features_for_boxplot = [f for f in key_features_for_boxplot if f in df_clustered_devices.columns]

    if key_features_for_boxplot:
        print("\nGenerating box plots for key features...")
        num_features_to_plot = len(key_features_for_boxplot)
        cols_subplot = 3 
        rows_subplot = (num_features_to_plot + cols_subplot - 1) // cols_subplot
        plt.figure(figsize=(15, rows_subplot * 5))
        for i, feature in enumerate(key_features_for_boxplot):
            plt.subplot(rows_subplot, cols_subplot, i + 1)
            sns.boxplot(x='cluster', y=feature, data=df_clustered_devices, palette='viridis')
            plt.title(f'{feature} by Cluster', fontsize=10)
        plt.tight_layout()
        # Save plot if needed: plt.savefig('box_plots_coffee_clusters.png')
        plt.show()
    else:
        print("Skipping box plots: no key features selected or available.")

    # Bar chart of mean feature values (from cluster_analysis)
    if not cluster_analysis.empty:
        print("\nGenerating bar chart of mean features...")
        cluster_analysis.T.plot(kind='bar', figsize=(18, 10), colormap='viridis') # Wider for more features
        plt.title('Mean Feature Values by Cluster (Original Scale)')
        plt.ylabel('Mean Value')
        plt.xticks(rotation=45, ha='right')
        plt.legend(title='Cluster')
        plt.tight_layout()
        # Save plot if needed: plt.savefig('bar_chart_mean_coffee_features.png')
        plt.show()
    else:
        print("Skipping bar chart: cluster_analysis is empty.")
else:
    print("\ndf_clustered_devices DataFrame not found. Skipping cluster analysis and visualization.")

print("\n--- Coffee Clustering Notebook Execution Finished ---")